In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    
    # use KNNImputer
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else SimpleImputer(missing_values=np.nan, strategy='median')
    
    X_norma_fixed = imputer.fit_transform(X_norma)
    
    return X_norma_fixed

# def remove_outliers(X, y):
#     print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
#     iforest = IsolationForest(max_samples=200, random_state=1, contamination='auto')
#     iforest.fit(X)
#     iforest_outlier_pred = iforest.predict(X)

#     mask = (iforest_outlier_pred!=-1)
#     X , y = X[mask, :], y[mask]
#     print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
#     return X, y

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    X , y = X[mask, :], y[mask]
    print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return X, y

def select_features(X, y, X_test, feature_num=50):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [3]:
X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Imputation of Missing Values
* [Reference](https://scikit-learn.org/stable/modules/impute.html)
* We use median of column instead of mean

In [4]:
X_train_missing_indices = X_train[X_train==np]
X_train = fill_missing_values(X_train, n_neighbors=75)
X_test = fill_missing_values(X_test)
print(X_train.shape)
print(X_test.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


(1212, 828)
(776, 828)


## 2. Outlier Detection
* [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)

In [5]:
X_train,y_train = remove_outliers(X_train,y_train)

IsolationForest-Traing data shape before removed: (1212, 828)
IsolationForest-Traing data shape after removed: (1200, 828)


## 3. Feature Selection

In [6]:
# X, X_test = feature_reduction(X, X_test,750)
X_train, X_test = select_features(X_train, y_train, X_test,feature_num = 50)
print("Traing data shape after selection: {}".format(X_train.shape))
print("Testing data shape after selection: {}".format(X_test.shape))

Traing data shape after selection: (1200, 50)
Testing data shape after selection: (776, 50)


## 4. Gaussian Process

In [7]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred(degree, X_train, y_train, X_val, y_val, X_test):
    kernel = C()*RBF()
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_train, y_train)
    gpr.score(X_train, y_train)
    
    y_pred = gbm.predict(X_test) 
    
    return y_pred

def train_k_fold(X, y, fold_num=10):
    kf = KFold(n_splits=fold_num, random_state=None, shuffle=False)
    kf.get_n_splits(X)
    test_score = 0.0
    
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        y_pred = fit_model_and_pred(1, X_train, y_train, X_val, y_val, X_val)
        score = r2_score(y_val, y_pred)

        print('The obtained validation r2 score is : ',score)
        test_score += score
    print("Validation score: %f"%(test_score/fold_num))
    
def train_k_fold_predict(X, y,X_test, fold_num=10):
    kf = KFold(n_splits=fold_num, random_state=None, shuffle=False)
    kf.get_n_splits(X)
    y_test_predict = np.zeros(X_test.shape[0])
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        y_pred = fit_model_and_pred(1, X_train, y_train, X_val, y_val, X_test)
        y_test_predict += y_pred

    return y_test_predict/fold_num

In [8]:
train_k_fold(X_train,y_train, fold_num=10) 

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001251 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l2: 90.305	training's 

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[16]	training's l2: 44.195	training's r2: 0.538611	valid_1's l2: 43.7711	valid_1's r2: 0.48456
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[17]	training's l2: 42.6167	training's r2: 0.555089	valid_1's l2: 43.1118	valid_1's r2: 0.492323
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[18]	training's l2: 41.1187	training's r2: 0.570728	valid_1's l2: 42.0106	valid_1's r2: 0.505291
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[19]	training's l2: 39.6165	training's r2: 0.586411	valid_1's l2: 40.9009	valid_1's r2: 0.518358
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[20]	training's l2: 38.3606	training's r2: 0.599522	valid_1's l2: 39.9956	valid_1's r2: 0.529018
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[21]	training's l2: 37.1758	training's r2: 0.611891	valid_1's l2: 3

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[111]	training's l2: 8.30541	training's r2: 0.913293	valid_1's l2: 24.9514	valid_1's r2: 0.706177
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[112]	training's l2: 8.2302	training's r2: 0.914078	valid_1's l2: 24.9531	valid_1's r2: 0.706156
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[113]	training's l2: 8.14873	training's r2: 0.914929	valid_1's l2: 24.9491	valid_1's r2: 0.706204
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[114]	training's l2: 8.05791	training's r2: 0.915877	valid_1's l2: 24.9745	valid_1's r2: 0.705904
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[115]	training's l2: 7.96078	training's r2: 0.916891	valid_1's l2: 24.9355	valid_1's r2: 0.706364
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[116]	training's l2: 7.86486	training's r2: 0.917892	valid_1'

[174]	training's l2: 5.05411	training's r2: 0.947236	valid_1's l2: 24.3865	valid_1's r2: 0.712828
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[175]	training's l2: 5.01593	training's r2: 0.947635	valid_1's l2: 24.3463	valid_1's r2: 0.713302
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[176]	training's l2: 4.98126	training's r2: 0.947996	valid_1's l2: 24.3074	valid_1's r2: 0.713761
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[177]	training's l2: 4.93114	training's r2: 0.94852	valid_1's l2: 24.3039	valid_1's r2: 0.713802
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[178]	training's l2: 4.91126	training's r2: 0.948727	valid_1's l2: 24.2894	valid_1's r2: 0.713972
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[179]	training's l2: 4.88535	training's r2: 0.948998	valid_1's l2: 24.3116	valid_1's r2: 0.713711
[LightGBM] [Warning] No further splits

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[211]	training's l2: 3.27642	training's r2: 0.965423	valid_1's l2: 42.4744	valid_1's r2: 0.556279
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[212]	training's l2: 3.2426	training's r2: 0.96578	valid_1's l2: 42.4719	valid_1's r2: 0.556306
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[213]	training's l2: 3.22245	training's r2: 0.965993	valid_1's l2: 42.4297	valid_1's r2: 0.556747
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[214]	training's l2: 3.20992	training's r2: 0.966125	valid_1's l2: 42.4331	valid_1's r2: 0.556711
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[215]	training's l2: 3.18843	training's r2: 0.966352	valid_1's l2: 42.3931	valid_1's r2: 0.557129
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[216]	training's l2: 3.17396	training's r2: 0.966505	valid_1's

[291]	training's l2: 2.02362	training's r2: 0.978644	valid_1's l2: 41.719	valid_1's r2: 0.564171
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[292]	training's l2: 2.01752	training's r2: 0.978709	valid_1's l2: 41.6987	valid_1's r2: 0.564383
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[293]	training's l2: 2.00947	training's r2: 0.978794	valid_1's l2: 41.7448	valid_1's r2: 0.563901
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[294]	training's l2: 1.99469	training's r2: 0.97895	valid_1's l2: 41.7595	valid_1's r2: 0.563748
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[295]	training's l2: 1.98874	training's r2: 0.979013	valid_1's l2: 41.7589	valid_1's r2: 0.563754
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[296]	training's l2: 1.97977	training's r2: 0.979107	valid_1's l2: 41.7547	valid_1's r2: 0.563798
[LightGBM] [Warning] No further splits 

[373]	training's l2: 1.34592	training's r2: 0.985796	valid_1's l2: 41.7019	valid_1's r2: 0.564349
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[374]	training's l2: 1.34148	training's r2: 0.985843	valid_1's l2: 41.698	valid_1's r2: 0.564391
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[375]	training's l2: 1.33052	training's r2: 0.985959	valid_1's l2: 41.687	valid_1's r2: 0.564506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[376]	training's l2: 1.32459	training's r2: 0.986021	valid_1's l2: 41.6725	valid_1's r2: 0.564657
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[377]	training's l2: 1.31889	training's r2: 0.986082	valid_1's l2: 41.6601	valid_1's r2: 0.564786
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[378]	training's l2: 1.31257	training's r2: 0.986148	valid_1's l2: 41.6284	valid_1's r2: 0.565118
[LightGBM] [Warning] No further splits 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[458]	training's l2: 0.92119	training's r2: 0.990279	valid_1's l2: 41.3112	valid_1's r2: 0.568432
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[459]	training's l2: 0.916119	training's r2: 0.990332	valid_1's l2: 41.271	valid_1's r2: 0.568851
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[460]	training's l2: 0.913849	training's r2: 0.990356	valid_1's l2: 41.2513	valid_1's r2: 0.569058
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[461]	training's l2: 0.909513	training's r2: 0.990402	valid_1's l2: 41.2628	valid_1's r2: 0.568937
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[462]	training's l2: 0.906799	training's r2: 0.99043	valid_1's l2: 41.2386	valid_1's r2: 0.569189
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[463]	training's l2: 0.903922	training's r2: 0.990461	vali

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[517]	training's l2: 0.718845	training's r2: 0.992414	valid_1's l2: 41.4979	valid_1's r2: 0.56648
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[518]	training's l2: 0.717389	training's r2: 0.992429	valid_1's l2: 41.483	valid_1's r2: 0.566636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[519]	training's l2: 0.71391	training's r2: 0.992466	valid_1's l2: 41.485	valid_1's r2: 0.566615
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[520]	training's l2: 0.711155	training's r2: 0.992495	valid_1's l2: 41.4826	valid_1's r2: 0.56664
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[521]	training's l2: 0.706781	training's r2: 0.992541	valid_1's l2: 41.4818	valid_1's r2: 0.566649
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[522]	training's l2: 0.702735	training's r2: 0.992584	valid_

In [9]:
Y_test_pred = train_k_fold_predict(X_train, y_train,X_test)
final_res = np.vstack((indices_test, Y_test_pred)).T

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l2: 90.305	training's 

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	training's l2: 76.7171	training's r2: 0.199086	valid_1's l2: 70.9076	valid_1's r2: 0.165004
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	training's l2: 72.7215	training's r2: 0.2408	valid_1's l2: 67.2939	valid_1's r2: 0.207558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	training's l2: 69.0243	training's r2: 0.279398	valid_1's l2: 64.0922	valid_1's r2: 0.245261
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	training's l2: 65.5986	training's r2: 0.315162	valid_1's l2: 61.1348	valid_1's r2: 0.280087
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[8]	training's l2: 62.5659	training's r2: 0.346822	valid_1's l2: 58.4506	valid_1's r2: 0.311696
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9]	training's l2: 59.6843	training's r2: 0.376906	valid_1's l2: 55.8303

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	training's l2: 15.2243	training's r2: 0.841061	valid_1's l2: 26.5703	valid_1's r2: 0.687113
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[61]	training's l2: 15.0059	training's r2: 0.843341	valid_1's l2: 26.4979	valid_1's r2: 0.687965
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[62]	training's l2: 14.7983	training's r2: 0.845508	valid_1's l2: 26.4253	valid_1's r2: 0.68882
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[63]	training's l2: 14.6095	training's r2: 0.847479	valid_1's l2: 26.406	valid_1's r2: 0.689047
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[64]	training's l2: 14.3759	training's r2: 0.849918	valid_1's l2: 26.3305	valid_1's r2: 0.689936
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[65]	training's l2: 14.1784	training's r2: 0.85198	valid_1's l2: 26

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[151]	training's l2: 5.99629	training's r2: 0.9374	valid_1's l2: 24.769	valid_1's r2: 0.708325
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[152]	training's l2: 5.95623	training's r2: 0.937818	valid_1's l2: 24.7575	valid_1's r2: 0.708459
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[153]	training's l2: 5.90009	training's r2: 0.938404	valid_1's l2: 24.7259	valid_1's r2: 0.708831
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[154]	training's l2: 5.84142	training's r2: 0.939017	valid_1's l2: 24.7509	valid_1's r2: 0.708537
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[155]	training's l2: 5.79581	training's r2: 0.939493	valid_1's l2: 24.7424	valid_1's r2: 0.708638
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[156]	training's l2: 5.74395	training's r2: 0.940034	valid_1's 

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[273]	training's l2: 2.24596	training's r2: 0.976298	valid_0's l2: 41.9455	valid_0's r2: 0.561805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[274]	training's l2: 2.23555	training's r2: 0.976408	valid_0's l2: 41.9006	valid_0's r2: 0.562274
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[275]	training's l2: 2.22472	training's r2: 0.976522	valid_0's l2: 41.8435	valid_0's r2: 0.56287
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[276]	training's l2: 2.2103	training's r2: 0.976674	valid_0's l2: 41.8334	valid_0's r2: 0.562976
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[277]	training's l2: 2.19747	training's r2: 0.97681	valid_0's l2: 41.8396	valid_0's r2: 0.562911
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[278]	training's l2: 2.18483	training's r2: 0.976943	valid_0's 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[339]	training's l2: 1.62188	training's r2: 0.982884	valid_0's l2: 41.9578	valid_0's r2: 0.561676
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[340]	training's l2: 1.61624	training's r2: 0.982944	valid_0's l2: 41.9345	valid_0's r2: 0.56192
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[341]	training's l2: 1.60538	training's r2: 0.983058	valid_0's l2: 41.9157	valid_0's r2: 0.562116
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[342]	training's l2: 1.59583	training's r2: 0.983159	valid_0's l2: 41.8951	valid_0's r2: 0.562331
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[343]	training's l2: 1.58692	training's r2: 0.983253	valid_0's l2: 41.8906	valid_0's r2: 0.562378
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[344]	training's l2: 1.57502	training's r2: 0.983379	valid_0'

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[412]	training's l2: 1.12322	training's r2: 0.988146	valid_0's l2: 41.5189	valid_0's r2: 0.566262
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[413]	training's l2: 1.11823	training's r2: 0.988199	valid_0's l2: 41.4816	valid_0's r2: 0.566651
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[414]	training's l2: 1.1134	training's r2: 0.98825	valid_0's l2: 41.4708	valid_0's r2: 0.566764
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[415]	training's l2: 1.10978	training's r2: 0.988288	valid_0's l2: 41.486	valid_0's r2: 0.566605
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[416]	training's l2: 1.10609	training's r2: 0.988327	valid_0's l2: 41.4774	valid_0's r2: 0.566695
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[417]	training's l2: 1.10257	training's r2: 0.988364	valid_0's 

[487]	training's l2: 0.812242	training's r2: 0.991428	valid_0's l2: 41.3584	valid_0's r2: 0.567939
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[488]	training's l2: 0.809813	training's r2: 0.991454	valid_0's l2: 41.3953	valid_0's r2: 0.567553
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[489]	training's l2: 0.807492	training's r2: 0.991478	valid_0's l2: 41.4253	valid_0's r2: 0.56724
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[490]	training's l2: 0.804315	training's r2: 0.991512	valid_0's l2: 41.4363	valid_0's r2: 0.567125
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[491]	training's l2: 0.798888	training's r2: 0.991569	valid_0's l2: 41.4547	valid_0's r2: 0.566932
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[492]	training's l2: 0.794022	training's r2: 0.991621	valid_0's l2: 41.4862	valid_0's r2: 0.566603
[LightGBM] [Warning] No further 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[560]	training's l2: 0.599753	training's r2: 0.993671	valid_0's l2: 41.6447	valid_0's r2: 0.564947
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[561]	training's l2: 0.596749	training's r2: 0.993702	valid_0's l2: 41.6184	valid_0's r2: 0.565222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[562]	training's l2: 0.594099	training's r2: 0.99373	valid_0's l2: 41.5893	valid_0's r2: 0.565526
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[563]	training's l2: 0.591655	training's r2: 0.993756	valid_0's l2: 41.5754	valid_0's r2: 0.565671
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[564]	training's l2: 0.588991	training's r2: 0.993784	valid_0's l2: 41.5868	valid_0's r2: 0.565552
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[565]	training's l2: 0.58725	training's r2: 0.993803	val

In [10]:
df_res = pd.DataFrame(final_res)
df_res.to_csv("our_result.csv", header = ["id", "y"], index=False)